### Importing the required libraries

In [1]:
# Python libraries
import pandas as pd
import numpy as np
from numpy import linalg as la

# Transformers libraries
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Model
import torch.nn.functional as F

### Definition of the model and the hook function

In [2]:
model_id = 'openai-community/gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, output_attentions=True)
gpt2_model = model.transformer

# Function to be called by the hook
output_list, module_list = [], []

def hook_fn(module, input, output):
    output_list.append(output)
    module_list.append(module)

# Attaching hook to all layers
for layer in model.modules():
    layer.register_forward_hook(hook_fn)


In [3]:
# Just a test to understand the last passages, leave it there for the moment
'''
prompt = "Explain this poem: O Captain! my Captain! our fearful trip is done,\nThe ship has weather’d every rack, the prize we sought is won,\nThe port is near, the bells I hear, the people all exulting,\nWhile follow eyes the steady keel, the vessel grim and daring;\nBut O heart! heart! heart!\nO the bleeding drops of red,\nWhere on the deck my Captain lies,\nFallen cold and dead.\n\nO Captain! my Captain! rise up and hear the bells;\nRise up—for you the flag is flung—for you the bugle trills,\nFor you bouquets and ribbon’d wreaths—for you the shores a-crowding,\nFor you they call, the swaying mass, their eager faces turning;\nHere Captain! dear father!\nThis arm beneath your head!\nIt is some dream that on the deck,\nYou’ve fallen cold and dead.\n\nMy Captain does not answer, his lips are pale and still,\nMy father does not feel my arm, he has no pulse nor will,\nThe ship is anchor’d safe and sound, its voyage closed and done,\nFrom fearful trip the victor ship comes in with object won;\nExult O shores, and ring O bells!\nBut I with mournful tread,\nWalk the deck my Captain lies,\nFallen cold and dead."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

with torch.no_grad():
    outputs = model(input_ids)
outputs.logits.shape
# Get the logits for the last token in the input sequence
logits = outputs.logits

# Select the logits for the last token
last_token_logits = logits[0, -1, :]

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)

# Get the predicted next token (the one with the highest probability)
predicted_token_id = torch.argmax(probabilities).item()

# Convert the predicted token ID back to a token
predicted_token = tokenizer.decode([predicted_token_id])

print(f"Predicted next token: ",predicted_token)
predicted_token_id

'''

'\nprompt = "Explain this poem: O Captain! my Captain! our fearful trip is done,\nThe ship has weather’d every rack, the prize we sought is won,\nThe port is near, the bells I hear, the people all exulting,\nWhile follow eyes the steady keel, the vessel grim and daring;\nBut O heart! heart! heart!\nO the bleeding drops of red,\nWhere on the deck my Captain lies,\nFallen cold and dead.\n\nO Captain! my Captain! rise up and hear the bells;\nRise up—for you the flag is flung—for you the bugle trills,\nFor you bouquets and ribbon’d wreaths—for you the shores a-crowding,\nFor you they call, the swaying mass, their eager faces turning;\nHere Captain! dear father!\nThis arm beneath your head!\nIt is some dream that on the deck,\nYou’ve fallen cold and dead.\n\nMy Captain does not answer, his lips are pale and still,\nMy father does not feel my arm, he has no pulse nor will,\nThe ship is anchor’d safe and sound, its voyage closed and done,\nFrom fearful trip the victor ship comes in with objec

### Choosing the prompt and running gpt2

In [3]:
# About life (Tokens: 7)
#prompt = "What's the meaning of life?"

# About Walt Whitman (Tokens: 308)
#prompt = "Explain this poem: O Captain! my Captain! our fearful trip is done,\nThe ship has weather’d every rack, the prize we sought is won,\nThe port is near, the bells I hear, the people all exulting,\nWhile follow eyes the steady keel, the vessel grim and daring;\nBut O heart! heart! heart!\nO the bleeding drops of red,\nWhere on the deck my Captain lies,\nFallen cold and dead.\n\nO Captain! my Captain! rise up and hear the bells;\nRise up—for you the flag is flung—for you the bugle trills,\nFor you bouquets and ribbon’d wreaths—for you the shores a-crowding,\nFor you they call, the swaying mass, their eager faces turning;\nHere Captain! dear father!\nThis arm beneath your head!\nIt is some dream that on the deck,\nYou’ve fallen cold and dead.\n\nMy Captain does not answer, his lips are pale and still,\nMy father does not feel my arm, he has no pulse nor will,\nThe ship is anchor’d safe and sound, its voyage closed and done,\nFrom fearful trip the victor ship comes in with object won;\nExult O shores, and ring O bells!\nBut I with mournful tread,\nWalk the deck my Captain lies,\nFallen cold and dead."

# About cats (Tokens: 559)
#prompt = "Cats, the enigmatic and beloved companions of humans for thousands of years, have always held a special place in our hearts and homes. With their graceful movements, expressive eyes, and independent spirits, cats captivate us with their unique charm and personalities. Domestic cats, scientifically known as Felis catus, are descendants of wild cats that were domesticated by ancient civilizations. While their exact origins remain shrouded in mystery, it is believed that cats first began living alongside humans in agricultural societies, where they proved invaluable in controlling rodent populations that threatened food stores. One of the most striking features of cats is their remarkable agility and athleticism. With their flexible bodies, keen reflexes, and razor-sharp claws, cats are natural-born hunters capable of stalking prey with unparalleled precision. Whether prowling through tall grasses in search of mice or pouncing on a feather toy in the living room, cats exhibit a predatory instinct that harkens back to their wild ancestry. Beyond their hunting prowess, cats are also renowned for their unique vocalizations and communicative behaviors. From the plaintive meow of a hungry kitten to the contented purr of a cat curled up in its favorite spot, felines use a diverse array of sounds and body language to express their needs, emotions, and desires. Each cat develops its own distinct vocal repertoire and personality, endearing themselves to their human companions with their individual quirks and mannerisms. Cats are also revered for their mysterious and independent nature. Unlike dogs, which often crave constant attention and companionship, cats are solitary creatures that value their autonomy and personal space. While they may form strong bonds with their human caregivers, cats also enjoy exploring the world at their own pace, indulging in moments of solitary contemplation, and retreating to quiet corners for rest and relaxation. In addition to their role as beloved pets, cats have also been revered and mythologized in cultures around the world. Ancient Egyptians worshipped cats as symbols of grace, fertility, and protection, with the goddess Bastet often depicted as a lioness or domestic cat. In Japanese folklore, the beckoning cat or 'Maneki-neko' is believed to bring good luck and fortune to its owner, while in European folklore, cats were sometimes associated with witchcraft and superstition. Today, cats continue to enchant and inspire us with their beauty, intelligence, and innate curiosity. Whether lounging in sunbeams, engaging in playful antics, or simply offering a comforting presence during times of need, cats have a way of leaving an indelible mark on our lives and reminding us of the simple joys of companionship and connection. As we celebrate these extraordinary creatures, let us cherish the special bond we share with our feline friends and honor the timeless legacy of the cat throughout history."

# About physics and music (Tokens: 997)
prompt = "Douglas Hofstadter’s “Gödel, Escher, Bach: An Eternal Golden Braid” (often abbreviated as GEB) is a unique and profound exploration of the interplay between mathematics, art, and music, presented through the lenses of three remarkable figures: the logician Kurt Gödel, the artist M.C. Escher, and the composer Johann Sebastian Bach. This Pulitzer Prize-winning work weaves together themes from various disciplines, creating a tapestry that examines the nature of human cognition, the essence of creativity, and the fundamental principles of self-reference and formal systems. Gödel’s Incompleteness Theorems: Kurt Gödel, an Austrian logician, is perhaps best known for his incompleteness theorems. These theorems fundamentally changed the landscape of mathematics and logic in the 20th century. Gödel’s first incompleteness theorem states that in any sufficiently powerful formal system, there are propositions that cannot be proven or disproven within the system. In other words, there are true statements that the system cannot prove. This theorem implies a limitation on the power of formal mathematical systems, revealing that there are truths that transcend formal proof. Hofstadter delves deeply into Gödel’s proof, illustrating how it employs a form of self-reference, similar to the paradoxes of Epimenides and the Liar. Gödel’s ingenious method involved encoding statements about the system within the system itself, leading to a statement that essentially says, “This statement is unprovable within this system.” If the system could prove this statement, it would lead to a contradiction, thereby establishing the truth of the statement outside the system’s proving capabilities. Escher’s Artistic Paradoxes: M.C. Escher, a Dutch graphic artist, is renowned for his mathematically inspired woodcuts, lithographs, and mezzotints, which feature impossible constructions, explorations of infinity, and intricate tessellations. Escher’s artwork often plays with perspective and optical illusion, creating images that defy conventional logic and evoke a sense of wonder and curiosity. Hofstadter draws parallels between Escher’s visual paradoxes and Gödel’s logical paradoxes. Escher’s works like “Drawing Hands,” where two hands are drawing each other, or “Relativity,” where multiple gravity-defying staircases coexist, exemplify the concept of self-reference and recursive structures. These artworks challenge our perception of reality and force us to reconsider the boundaries between the possible and the impossible. Bach’s Musical Patterns: Johann Sebastian Bach, the German composer and musician of the Baroque period, is celebrated for his intricate compositions and mastery of counterpoint. Hofstadter examines Bach’s use of fugues, canons, and other forms of musical recursion to illustrate patterns and structures that resonate with the themes of self-reference and formal systems. Bach’s music, with its interweaving melodies and harmonious complexity, serves as a metaphor for the layered and recursive nature of formal systems. Hofstadter particularly highlights Bach’s “Musical Offering,” a collection of compositions based on a single musical theme that is transformed and developed in various ingenious ways. This mirrors the idea of taking a simple formal system and exploring its possibilities through recursive applications and transformations. The Concept of Strange Loops: A central theme in GEB is the concept of “strange loops,” a term Hofstadter uses to describe a hierarchical structure that loops back on itself. This idea is illustrated through Gödel’s theorems, Escher’s artwork, and Bach’s music. A strange loop occurs when, by moving through different levels of a system, one unexpectedly arrives back at the starting point. Hofstadter argues that strange loops are at the core of consciousness and self-awareness. He suggests that the human mind is a complex, recursive system capable of self-reference, and this ability to reflect upon itself is what gives rise to the phenomenon of “I.” Our thoughts can contemplate themselves, creating a loop of self-reference that is both a source of paradox and a hallmark of intelligence. Formal Systems and Symbolic Representation: Hofstadter explores the nature of formal systems, which are systems of rules and symbols used to produce statements and derive conclusions. He discusses how formal systems can represent complex ideas and how they relate to the way we think and communicate. The notion of symbolic representation is crucial in understanding how abstract concepts can be encoded and manipulated within a formal system. The book frequently uses dialogues between fictional characters, such as Achilles and the Tortoise, to explore these concepts in an accessible and engaging manner."



input_ids = tokenizer(prompt, return_tensors="pt").input_ids

with torch.no_grad():
    outputs = model(input_ids)
    
for i, module in enumerate(module_list):
    try:
        print(i, module, '   output_shape: ', output_list[i].shape)
    except:
        try:
            print(i, module, '   output_shape: ', len(output_list[i]))
        except:
            print(i, module)

attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
attn_output shape:  torch.Size([1, 308, 768])
0 Embedding(50257, 768)    output_shape:  torch.Size([1, 308, 768])
1 Embedding(1024, 768)    output_shape:  torch.Size([1, 308, 768])
2 Dropout(p=0.1, inplace=False)    output_shape:  torch.Size([1, 308, 768])
3 LayerNorm((768,), eps=1e-05, elementwise_affine=True)    output_shape:  torch.Size([1, 308, 768])
4 Conv1D()    output_shape:  torch.Size([1, 308, 2304])
5 Dropout(p=0.1, inplace=False)    output_shape:  torch.Size([1, 12, 308, 308])
6

### Storing output list and module list

#### Word2Vec conversion

Remember that the passages are:
    
    1. Convert word to tokens and tokens to vectors
    2. Apply the positional encoding matrix
    3. Sum the positional encoding to the vectorial representation of the prompt

In [4]:
Token2Vec          = output_list[0][0]
PositionalEncoding = output_list[1][0]
PositionPlusVect   = output_list[2][0]

torch.save(Token2Vec, "output_Captain/word2vec/Token2Vec.pt")
torch.save(PositionalEncoding, "output_Captain/word2vec/PositionalEncoding.pt")
torch.save(PositionPlusVect, "output_Captain/word2vec/PositionPlusVect.pt")

#### Storing evolution of the decoder blocks

In [5]:
index_list = np.array([3, 4, 5, 7, 9, 11, 13, 15])
'''
Decoder_01_FirstNormalization  = output_list[3]
Decoder_01_QKV_representation  = output_list[4]
Decoder_01_AttentionHeads      = output_list[5]
Decoder_01_AttentionProj       = output_list[7]
Decoder_01_SecondNormalization = output_list[9]
Decoder_01_FirstLayerNN        = output_list[11]
Decoder_01_SecondLayerNN       = output_list[13]  'Delta space'
Decoder_01_final_output        = output_list[15]  'Residual + Delta space'
'''
module_name = ["FirstNormalization", "QKV_representation", "AttentionHeads", "AttentionProj", "SecondNormalization", "FirstLayerNN", "SecondLayerNN", "Decoder_Final_Output"]
# Create Decoder_mask and flatten it
Decoder_mask = np.concatenate([index_list + i*13 for i in range(12)])

# Assuming output_list is already defined, we can proceed
# Extract elements for Decoder_list
Decoder_list = [output_list[mask] for mask in Decoder_mask]

PositionalEmbedding = output_list[2]

Decoder_list = [output_list[mask] for mask in Decoder_mask]

for i in range(12):
    for j in range(8):
        print("Decoder ", i + 1 , " ", module_name[j])
        torch.save(Decoder_list[j+i*8], "output_Captain/decoder/decoder_"+str(i+1) +"/"+module_name[j]+".pt")

Decoder  1   FirstNormalization
Decoder  1   QKV_representation
Decoder  1   AttentionHeads
Decoder  1   AttentionProj
Decoder  1   SecondNormalization
Decoder  1   FirstLayerNN
Decoder  1   SecondLayerNN
Decoder  1   Decoder_Final_Output
Decoder  2   FirstNormalization
Decoder  2   QKV_representation
Decoder  2   AttentionHeads
Decoder  2   AttentionProj
Decoder  2   SecondNormalization
Decoder  2   FirstLayerNN
Decoder  2   SecondLayerNN
Decoder  2   Decoder_Final_Output
Decoder  3   FirstNormalization
Decoder  3   QKV_representation
Decoder  3   AttentionHeads
Decoder  3   AttentionProj
Decoder  3   SecondNormalization
Decoder  3   FirstLayerNN
Decoder  3   SecondLayerNN
Decoder  3   Decoder_Final_Output
Decoder  4   FirstNormalization
Decoder  4   QKV_representation
Decoder  4   AttentionHeads
Decoder  4   AttentionProj
Decoder  4   SecondNormalization
Decoder  4   FirstLayerNN
Decoder  4   SecondLayerNN
Decoder  4   Decoder_Final_Output
Decoder  5   FirstNormalization
Decoder  5  

Let's extract also the intermediate result of Output Attention + Residual connection

In [6]:
# Extract also Output Attention + Residual connection

SecondLayerNN_list         =  [torch.load(f"output_Captain/decoder/decoder_{i+1}/SecondLayerNN.pt")[0] for i in range(12)]
Decoder_Final_Output_list  =  [torch.load(f"output_Captain/decoder/decoder_{i+1}/Decoder_Final_Output.pt")[0][0] for i in range(12)]

AttentionPlusResidual_list =  [DecOut - SecLayer  for DecOut, SecLayer in zip(Decoder_Final_Output_list, SecondLayerNN_list)]

for i, AttentionPlusResidual in enumerate(AttentionPlusResidual_list):
    torch.save((AttentionPlusResidual.unsqueeze(0),)
, f"output_Captain/decoder/decoder_{i+1}/AttentionPlusResidual.pt")

Push Back the last token to the language domain to study the evolution of the probability distribution during the 12 blocks

In [7]:
# Extract the final normalization layer and the 'inverse matrix'
ln_f = gpt2_model.ln_f
lm_head = model.lm_head

# Let's save the 'projection' on the vocabulary before and after the softmax
for i, (out_attention, out_decoder) in enumerate(zip(AttentionPlusResidual_list, Decoder_Final_Output_list)):
    print(f"Decoder {i+1}")
    
    final_norm = ln_f(out_attention)
    projection = lm_head(final_norm[-1])
    softmax = F.softmax(projection, dim=-1)
    
    top_values, top_indices = torch.topk(softmax, 10, dim=-1)

    print("    Top10 attn: ", top_indices, top_values)

    torch.save(projection, f"output_Captain/last_token_pdf/decoder_{i+1}/attention_projection.pt")
    torch.save(softmax, f"output_Captain/last_token_pdf/decoder_{i+1}/attention_softmax.pt")

    final_norm = ln_f(out_decoder)
    projection = lm_head(final_norm[-1])
    softmax = F.softmax(projection, dim=-1)

    torch.save(projection, f"output_Captain/last_token_pdf/decoder_{i+1}/out_decoder_projection.pt")
    torch.save(softmax, f"output_Captain/last_token_pdf/decoder_{i+1}/out_decoder_softmax.pt")
    
    # Extract the top 10 entries with highest softmax values
    top_values, top_indices = torch.topk(softmax, 10, dim=-1)
    print("     Top10 dec: ", top_indices, top_values)
    

Decoder 1
    Top10 attn:  tensor([ 383,  198,  317,  314, 2448,  554,  632,  843, 1081, 1649]) tensor([0.0799, 0.0761, 0.0451, 0.0286, 0.0252, 0.0192, 0.0190, 0.0171, 0.0164,
        0.0158], grad_fn=<TopkBackward0>)
     Top10 dec:  tensor([ 383,  198,  632,  843,  887, 1081, 1002,  554, 1318, 1649]) tensor([0.1186, 0.1049, 0.0713, 0.0696, 0.0537, 0.0327, 0.0299, 0.0293, 0.0271,
        0.0267], grad_fn=<TopkBackward0>)
Decoder 2
    Top10 attn:  tensor([ 383,  198,  843,  632,  887, 1649, 1081, 1002, 1318,  554]) tensor([0.0966, 0.0772, 0.0644, 0.0564, 0.0532, 0.0361, 0.0316, 0.0299, 0.0293,
        0.0236], grad_fn=<TopkBackward0>)
     Top10 dec:  tensor([ 632,  383,  198, 1002, 1649, 1318,  843,  887,  770, 1081]) tensor([0.0903, 0.0777, 0.0605, 0.0564, 0.0516, 0.0500, 0.0481, 0.0360, 0.0333,
        0.0257], grad_fn=<TopkBackward0>)
Decoder 3
    Top10 attn:  tensor([ 632,  198,  383, 1002, 1649,  843,  887, 1318,  770,  775]) tensor([0.0828, 0.0588, 0.0557, 0.0544, 0.0540, 0.05

In [9]:
# Template:
'''
ln_f = gpt2_model.ln_f
lm_head = model.lm_head

final_norm = ln_f(Decoder_Final_Output_list[-1])
# Get the output from the lm_head
projection_1 = lm_head(final_norm[-1])

# Apply softmax on projection_1
softmax_output = F.softmax(projection_1, dim=-1)
# Extract the top 10 entries with highest softmax values
top_values, top_indices = torch.topk(softmax_output, 10, dim=-1)

print("Top 10 values:")
print(top_values)

print("Top 10 indices:")
print(top_indices)

print(softmax_output)
#print(projection_1-)
diff = output_list[161][0][-1] - projection_1
diff_soft = softmax_output - F.softmax(output_list[161][0][-1], dim=-1)
places = np.ones(diff.shape)
'''

'\nln_f = gpt2_model.ln_f\nlm_head = model.lm_head\n\nfinal_norm = ln_f(Decoder_Final_Output_list[-1])\n# Get the output from the lm_head\nprojection_1 = lm_head(final_norm[-1])\n\n# Apply softmax on projection_1\nsoftmax_output = F.softmax(projection_1, dim=-1)\n# Extract the top 10 entries with highest softmax values\ntop_values, top_indices = torch.topk(softmax_output, 10, dim=-1)\n\nprint("Top 10 values:")\nprint(top_values)\n\nprint("Top 10 indices:")\nprint(top_indices)\n\nprint(softmax_output)\n#print(projection_1-)\ndiff = output_list[161][0][-1] - projection_1\ndiff_soft = softmax_output - F.softmax(output_list[161][0][-1], dim=-1)\nplaces = np.ones(diff.shape)\n'

In [10]:
print("Finished")

Finished
